In [41]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [1]:
import sys
sys.path.append('~/aiffel/aiffelthon')

In [ ]:
!ls

In [1]:
!pip install rouge_score
!pip install datasets==1.0.2
!pip install transformers==4.24.0
!pip install transformer-utils
!pip install packaging

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=4976236e56a9496b7b4a3bb94cfafe11b711c1e9a80210b1ff404b856722dbf6
  Stored in directory: /aiffel/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
     |████████████████████████████████| 1.8 MB 6.0 MB/s            
  Attempting uninstall: datasets
    Found existing installation: datasets 1.14.0
    Uninstalling datasets-1.14.0:
      Successfully uninstalled datasets-1.14.0
     |████████████████████████████████| 5.5 MB 6.2 MB/s            
     |████████████████████████████████| 7.6 MB 68.2 MB/s            
     |████████████████████████████████| 163 kB 101.7 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: huggingface-hub
    Found 

     |████████████████████████████████| 86 kB 10.5 MB/s            


In [2]:
# 필요한 라이브러리 불러오기
import datasets
from datasets import load_dataset
from datasets import Dataset
import transformers
import pandas as pd
import re
import random
from IPython.display import display, HTML

#Tokenizer
from transformers import BertTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

#Tokenizer, Encoder-Decoder Model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
#Training
from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from seq2seq_training_args import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [ ]:
# 경로 지정
#%cd ~/aiffel/aiffelthon

In [ ]:
#!unzip ~/aiffel/aiffelthon/csv.zip

In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_trainer.py

In [ ]:
#!wget https://github.com/huggingface/transformers/blob/main/src/transformers/utils/import_utils.py

In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/models/encoder_decoder/modeling_encoder_decoder.py

In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/configuration_utils.py

In [3]:
# 데이터 불러오기
train_df = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train_Sum1.csv')
#train_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
train_df.rename(columns = {"input": "Text"}, inplace = True)
train_df.rename(columns = {"sentence_onesent": "Summary"}, inplace = True)
#train_df.rename(columns = {"sentence_per_20": "Summary"}, inplace = True)

val_df = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/val_Sum1.csv')
#val_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
val_df.rename(columns = {"input": "Text"}, inplace = True)
val_df.rename(columns = {"summary_onesent": "Summary"}, inplace = True)
#val_df.rename(columns = {"summary_per_20": "Summary"}, inplace = True)
print(len(train_df), len(val_df))

125690 18300


In [ ]:
train_df.head()

In [ ]:
val_df.head()

In [ ]:
# 10번째 row만 추출
# train_df = train_df.iloc[range(0, len(train_df), 2)]
train_df = train_df.iloc[::2, :]
#val_df = val_df.iloc[range(0, len(val_df), 2)]
print(len(train_df), len(val_df))

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거
    sentence = re.sub('"','', sentence) # 쌍따옴표 제거
    sentence = re.sub("'",'', sentence) # 따옴표 제거
    sentence = re.sub('\n','', sentence) # \n " 제거
    sentence = re.sub('.{2,3}\W{0,1}기자','', sentence) # 기자 이름 제거
    sentence = re.sub(r'[?.!,][/?.!,]', '', sentence) # 여러개 문장 부호를 하나의 문장부호로 바꿉니다
    sentence = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣a-z0-9]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [5]:
# 전체 Text 데이터에 대한 전처리 (1)
from tqdm import tqdm
clean_text = []

for s in tqdm(train_df['Text']):
    clean_text.append(preprocess_sentence(s))
    
train_df['Text'] = clean_text

100%|██████████| 125690/125690 [00:26<00:00, 4766.27it/s]


In [6]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines = []

for s in tqdm(train_df['Summary']):
      clean_headlines.append(preprocess_sentence(s))
        
train_df['Summary'] = clean_headlines

100%|██████████| 125690/125690 [00:03<00:00, 38312.81it/s]


In [7]:
# 전체 Text 데이터에 대한 전처리 (1)

clean_text_val = []

for s in tqdm(val_df['Text']):
    clean_text_val.append(preprocess_sentence(s))
    
val_df['Text'] = clean_text_val

100%|██████████| 18300/18300 [00:03<00:00, 4842.50it/s]


In [8]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines_val = []

for s in tqdm(val_df['Summary']):
      clean_headlines_val.append(preprocess_sentence(s))
        
val_df['Summary'] = clean_headlines_val

100%|██████████| 18300/18300 [00:00<00:00, 36627.35it/s]


In [ ]:
train_df.head()

In [ ]:
val_df.head()

In [ ]:
# reset_index 사용
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [21]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
#val_len = len(val_df) // 2
val_data = Dataset.from_pandas(val_df.iloc[::2, :])
test_data = Dataset.from_pandas(val_df.iloc[1::2, :])

In [10]:
print(train_data)
print(val_data)
print(test_data)

Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 125690)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 9150)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 9150)


In [15]:
#tokenizer = BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")
#parameter setting
encoder_max_length=512
decoder_max_length=64
batch_size = 16

In [11]:
model_checkpoint = 'kykim/bertshared-kor-base'

In [18]:
tokenizer = BertTokenizerFast.from_pretrained('kykim/bertshared-kor-base')

In [15]:
tokenizer("안녕하세요 처음 뵙겠습니다 !")

{'input_ids': [2, 16453, 14121, 4542, 14301, 2005, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [17]:
max_input = 512
max_target = 128
batch_size = 16
model_checkpoint = "kykim/bertshared-kor-base"

In [16]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    # because RoBERTa automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [20]:
#train_data = train_data.select(range(32))

In [21]:
#val_data = val_data.select(range(8))

In [19]:
def preprocess_data(data_to_process):
    #get all the dialogues
    inputs = [dialogue for dialogue in data_to_process['Text']]
    #tokenize the dialogues
    model_inputs = tokenizer(inputs,  max_length=encoder_max_length, padding='max_length', truncation=True)
    #tokenize the summaries
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(data_to_process['Summary'], max_length=decoder_max_length, padding='max_length', truncation=True)

    #set labels
    model_inputs['labels'] = targets['input_ids']
    #return the tokenized data
    #input_ids, attention_mask and labels
    return model_inputs

In [22]:
#processing training data
train_data = train_data.map(
    preprocess_data, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])

train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],)

#processing validation data
val_data = val_data.map(
    preprocess_data, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])

val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],)

  0%|          | 0/7856 [00:00<?, ?ba/s]

  0%|          | 0/572 [00:00<?, ?ba/s]

In [24]:
train_data[0:1]

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
#processing training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])

train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],)

#processing validation data
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])

val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],)

In [19]:
#train_data = train_data.map(preprocess_data, batched = True, remove_columns=["Text", "Summary"])
#val_data = val_data.map(preprocess_data, batched = True, remove_columns=["Text", "Summary"])

/opt/conda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/126 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [25]:
# 인코더 모델 불러오기
from transformers import AutoModelForCausalLM
import torch
#bertshared =EncoderDecoderModel.from_pretrained("kykim/bertshared-kor-base")
#bertshared = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model = EncoderDecoderModel.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/589M [00:00<?, ?B/s]

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [24]:
model.save_pretrained(model_checkpoint)

In [26]:
model.config

EncoderDecoderConfig {
  "_commit_hash": "8ea27677d006a547aee2a7753d1ff18a0346860c",
  "_name_or_path": "kykim/bertshared-kor-base",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 2,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": 2,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "embedding_size": 768,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 3,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_drop

In [27]:
# set special tokens
#from transformers import EncoderDecoderConfig
#model.config.decoder_start_token_id = tokenizer.bos_token_id   
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
# set decoding params                               
model.config.max_length = 128
model.config.min_length = 32
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4
model.config.vocab_size = model.config.encoder.vocab_size

In [32]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

In [28]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),}

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

In [29]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [30]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/aiffel/aiffel/aiffelthon/ch/",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,  # demo
    per_device_eval_batch_size=8,
    learning_rate=3e-05,
    save_steps=500,
    eval_steps=500,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="/aiffel/aiffel/aiffelthon/logs",
    logging_steps=2000,
    save_total_limit=3,
    fp16 = True)

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/aiffel/aiffel/aiffelthon/ch/",
    num_train_epochs=1, 
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-05,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_steps=2000,  # set to 2000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=500,  # set to 7500 for full training
    warmup_steps=500,  # set to 3000 for full training
    logging_dir="/aiffel/aiffel/aiffelthon/logs",
    save_total_limit=3,
    fp16=True,)
    #max_steps=128)

In [31]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model = model, 
    args = training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

Using cuda_amp half precision backend


### DataCollatorForSeq2Seq의 역할
-  decoder 모델이 있어서 model이라는 인자를 추가하면, pretrained 모델이 `prepare_decoder_input_ids_from_labels`를 가지고 있는 경우에 `decoder_input_ids`를 만든다고 한다.

In [32]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 125690
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15712
  Number of trainable parameters = 147298320
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a meth

Step,Training Loss
2000,1.190100
4000,0.902800
6000,0.852300
8000,0.829000
10000,0.792600
12000,0.776700
14000,0.765600


Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch/checkpoint-500
Configuration saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-500/tokenizer_config.json
Special tokens file saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [/aiffel/aiffel/aiffelthon/ch/checkpoint-6000] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass t

Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch/checkpoint-4500
Configuration saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-4500/config.json
Model weights saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-4500/pytorch_model.bin
tokenizer config file saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-4500/tokenizer_config.json
Special tokens file saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-4500/special_tokens_map.json
Deleting older checkpoint [/aiffel/aiffel/aiffelthon/ch/checkpoint-3000] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to p

Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch/checkpoint-8000
Configuration saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-8000/config.json
Model weights saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-8000/pytorch_model.bin
tokenizer config file saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-8000/tokenizer_config.json
Special tokens file saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-8000/special_tokens_map.json
Deleting older checkpoint [/aiffel/aiffel/aiffelthon/ch/checkpoint-6500] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to p

tokenizer config file saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-11500/tokenizer_config.json
Special tokens file saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-11500/special_tokens_map.json
Deleting older checkpoint [/aiffel/aiffel/aiffelthon/ch/checkpoint-10000] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch/checkpoint-12000
Configuration saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-12000/config.json
Mo

Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch/checkpoint-15500
Configuration saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-15500/config.json
Model weights saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-15500/pytorch_model.bin
tokenizer config file saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-15500/tokenizer_config.json
Special tokens file saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-15500/special_tokens_map.json
Deleting older checkpoint [/aiffel/aiffel/aiffelthon/ch/checkpoint-14000] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no nee

TrainOutput(global_step=15712, training_loss=0.860140915072377, metrics={'train_runtime': 8198.0356, 'train_samples_per_second': 15.332, 'train_steps_per_second': 1.917, 'total_flos': 4.426768226893824e+16, 'train_loss': 0.860140915072377, 'epoch': 1.0})

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained('/aiffel/aiffel/aiffelthon/ch/checkpoint-15500/').to("cuda")
batch_size = 16

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=32, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])
pred_str = results["pred"]
label_str = results["Summary"]

loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/config.json
Model config EncoderDecoderConfig {
  "_commit_hash": "8ea27677d006a547aee2a7753d1ff18a0346860c",
  "_name_or_path": "kykim/bertshared-kor-base",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 2,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": 2,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "embedding_size": 768,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 3,
    "exponential_decay_le

loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/config.json
Model config EncoderDecoderConfig {
  "_commit_hash": "8ea27677d006a547aee2a7753d1ff18a0346860c",
  "_name_or_path": "kykim/bertshared-kor-base",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 2,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": 2,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "embedding_size": 768,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 3,
    "exponential_decay_le

loading weights file /aiffel/aiffel/aiffelthon/ch/checkpoint-15500/pytorch_model.bin
The following encoder weights were not tied to the decoder ['bert/pooler']
All model checkpoint weights were used when initializing EncoderDecoderModel.

All the weights of EncoderDecoderModel were initialized from the model checkpoint at /aiffel/aiffel/aiffelthon/ch/checkpoint-15500/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use EncoderDecoderModel for predictions without further training.
The following encoder weights were not tied to the decoder ['bert/pooler']
/opt/conda/lib/python3.9/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maxi

  0%|          | 0/572 [00:00<?, ?ba/s]

In [ ]:
trainer.evaluate()

In [ ]:
print(type(pred_str), type(label_str))

In [43]:
for num in range(5,4):
    print('predicted sentence : ',pred_str[num])
    print('real sentence : ', label_str[num])
    print('-'*100)

predicted sentence :  소비자의 외식 트렌드는 시시각각 변하고 변화에 맞게 신메뉴가 개발되고 신사업이 떴다가 사라지는 현상은 반복되며 소비자의 외식 소비형태 데이터 못지않게 향후 향후 향후 앞으로의 향후는 반복된다
real sentence :  농림식품부 주관으로 열린 2020년 떠오르는 외식 경향 발표의 핵심 키워드는 그린 오션 바이 미 포 미 멀티 스트리밍 편리미엄 외식의 확대이다
----------------------------------------------------------------------------------------------------


In [71]:
num = 2500
print('predicted sentence : ',pred_str[num])
print('\n')
print('real sentence : ', label_str[num])
print('\n')
print('Text', test_data[num])

predicted sentence :  손 식품의약품안전처장은 어패류 관련해서 우려들이 많은데 콜콜류 관련 우려들이 많은데 콜 콜 콜에 대한 우려들이 많다고 했다


real sentence :  손 식품의약품안전처장은 식중독이 발생하지 않게 어패류를 깨끗한 수돗물로 잘 세척해 오염된 부분을 제거하고 교차오염이 되지 않도록 손 등을 잘 씻는 행동요령을 잘 지켜달라고 했다


Text {'Text': '위원장 양승조 처장님도 한 말씀 하시지요 식품의약품안전처장 손문기 어패류 관련해서 좀 우려들이 많으신데 일단 콜레라라든지 비브리오 오염이 되더라도 깨끗한 수돗물로 잘 세척해서 그 부분을 제거하고 그다음에 손과 생선 칼 도마들이 교차오염 되지 않도록 잘 씻어서 하시면 식중독이나 이런 피해가 발생하지 않을 걸로 보고 있습니다 그래서 그런 행동요령을 잘 지켜 주시기 바라고 그래도 좀 의심되는 경우에는 잘 끓이고 익혀서 드시면 크게 문제는 없을 것으로 보고 있습니다 저희도 여름철 안전관리를 위해서 최선을 다하도록 하겠습니다 위원장 양승조 질병관리본부장님도 한 말씀 하세요 질병관리본부장 정기석 저희는 한 박자 빠른 선제 대응 그다음에 과하다 싶을 정도의 대응을 하는 것을 원칙으로 삼고 일을 열심히 하고 있습니다 그래서 앞으로 조금 더 신뢰받는 기관이 될 수 있도록 그렇게 해서 여러분들께서 그렇게 걱정을 많이 하시는 일이 없도록 더 철두철미하게 일하도록 하겠습니다 감사합니다 위원장 양승조 수고하셨습니다 서면질의하신 위원님이 계십니다 강석진 위원님 기동민 위원님 김명연 위원님 송석준 위원님 인재근 위원님 남인순 위원님 양승조 위원님 박인숙 위원님 정춘숙 위원님으로부터 서면질의서가 제출되었습니다 소상하고 성실한 답변서를 작성하여 9월 5일까지 제출해 주시기 바랍니다 서면질의와 답변 내용은 회의록에 게재하도록 하겠습니다 여러 위원님들 수고 많이 하셨습니다 특히 끝까지 남아 주신 인재근 위원님 김광수 위원님 최도자 위원님 김승희 위원님 윤소하 위원님 정말 수고 많으셨다는 존경과 감

In [ ]:
for num in range(0, len(label_str), 100):
    print('predicted sentence : ',pred_str[num])
    print('real sentence : ', label_str[num])
    print('-'*100)

In [35]:
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE L SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid)

ROUGE 1 SCORE:  Score(precision=0.08858293650615254, recall=0.10559894613583143, fmeasure=0.08848478563218844)
ROUGE 2 SCORE:  Score(precision=0.016476199405080792, recall=0.02023315118397086, fmeasure=0.016522825463291396)
ROUGE L SCORE:  Score(precision=0.08795556315180239, recall=0.10482220921155352, fmeasure=0.08784555763278679)
